In [1]:
%pip install catboost
%pip install xgboost
%pip install lightgbm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: xgboost in c:\users\joshl\anaconda3\lib\site-packages (3.0.0)




[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
import pandas as pd
import xgboost as xgb
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, confusion_matrix, classification_report
import numpy as np

In [12]:
# --- LOAD DATA ---
try:
    df = pd.read_csv('spotify_cleaned.csv')
 
    print("File loaded successfully.")
    df['pop_bucket']=df['pop_bucket'].apply(lambda x:x-1)
    # Bucket 'popularity'
    # df['popularity_bucket'] = pd.cut(
    #     df['popularity'],
    #     bins=[-1,0,40,60,85],
    #     labels=[1, 2, 3, 4]
    # ).astype(int)

    # print("Bucketing completed. Here's a sample:")
    # print(df[['popularity', 'popularity_bucket']].head())
    print(df.head(5))
except FileNotFoundError:
    print("Error: File 'spotify_cleaned.csv' not found.")
except pd.errors.EmptyDataError:
    print("Error: 'spotify_cleaned.csv' is empty.")
except pd.errors.ParserError:
    print("Error: Unable to parse 'spotify_cleaned.csv'. Check file format.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

File loaded successfully.
   Unnamed: 0.1  Unnamed: 0          artist_name         track_name  \
0          1044        1044  Menahan Street Band  Seven Is the Wind   
1       1130163     1424053            Flatsound         We'll Live   
2        927251     1121151              Subhira      Ruta del Inca   
3        275932      275932        Scary Pockets         Wonderwall   
4        834627      981623    Pain of Salvation        Iter Impius   

                 track_id  popularity  year     genre  danceability  energy  \
0  5qh2fad21W9se0d4IpKcyi          22  2012  afrobeat         0.525   0.577   
1  1XuAT2hAkjDvGqd4MMv2wL          16  2011       emo         0.339   0.624   
2  0ID7odbzAGY7C9cP2PzLRX           0  2007  afrobeat         0.552   0.766   
3  120tmueqvWTGdHET4u5sPP          36  2017      funk         0.756   0.732   
4  12kxpEvK9B99z02rNRmnSF          26  2004   swedish         0.437   0.584   

   ...  speechiness  acousticness  instrumentalness  liveness  valence  

Use a tree model like XGBoost. Better for tabular data and known features.

In [4]:
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression

In [15]:
# --- MODEL TRAINING ---
n_runs = 10
models = {
    "Random Forest": RandomForestClassifier(n_estimators=400, random_state=69),
    "XGBoost": xgb.XGBClassifier(
        objective='multi:softmax',
        num_class=4,
        eval_metric='mlogloss',
        use_label_encoder=False,
        random_state=69),
    "CatBoost": CatBoostClassifier(
        iterations=400,
        depth=8,
        learning_rate=0.1,
        loss_function='MultiClass',
        verbose=False,
        random_state=69),
    "Logistic Regression": LogisticRegression(
        max_iter=1000,
        multi_class='ovr',
        class_weight='balanced')
}
all_results = {model_name: [] for model_name in models.keys()}

for run in range(n_runs):
    print("Starting run", run + 1)
    df = df.sample(20000, random_state=run)

if 'df' in locals():
    try:
        X = df.drop(['popularity', 'pop_bucket'], axis=1)
        y = df['pop_bucket']

        # Identify and label encode categorical columns
        categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

        le = LabelEncoder()
        for col in categorical_cols:
            X[col] = le.fit_transform(X[col])

        print(f"Categorical columns encoded: {categorical_cols}")

        # Split data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


        results = {}

        # Train and evaluate
        for name, model in models.items():
            print(f"\nTraining {name}...")
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            acc = accuracy_score(y_test, y_pred)
            results[name] = acc
            all_results[name].append(acc)
            print("name:", name)

            print("Classification Report:")
            print(classification_report(y_test, y_pred))

            try:
                if hasattr(model, "feature_importances_"):
                    importances = model.feature_importances_
                elif hasattr(model, "coef_"):
                    importances = np.abs(model.coef_[0])  # For Logistic Regression
                else:
                    importances = None

                if importances is not None:
                    feature_importances = sorted(
                        zip(X_train.columns, importances), key=lambda x: x[1], reverse=True
                    )
                    print("\nFeature Importances:")
                    for feature, importance in feature_importances:
                        print(f"{feature}: {importance:.4f}")
                else:
                    print("Feature importances not available for this model.")
            except Exception as e:
                print(f"An error occurred while calculating feature importances: {e}")

    except KeyError as e:
        print(f"Error: Target column '{e}' not found in the DataFrame.")
    except Exception as e:
        print(f"An error occurred during model training or evaluation: {e}")

Starting run 1
Starting run 2
Starting run 3
Starting run 4
Starting run 5
Starting run 6
Starting run 7
Starting run 8
Starting run 9
Starting run 10
Categorical columns encoded: ['artist_name', 'track_name', 'track_id', 'genre', 'decade']

Training Random Forest...
name: Random Forest
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.50      0.03      0.05       183
           2       0.65      0.56      0.60      1383
           3       0.77      0.88      0.82      2433

    accuracy                           0.73      4000
   macro avg       0.48      0.37      0.37      4000
weighted avg       0.72      0.73      0.71      4000


Feature Importances:
genre: 0.0754
Unnamed: 0: 0.0739
Unnamed: 0.1: 0.0738
duration_ms: 0.0644
instrumentalness: 0.0589
danceability: 0.0579
loudness: 0.0545
acousticness: 0.0540
energy: 0.0536
speechiness: 0.0503
year: 0.0489
track_name: 0.0479
valence

C:\Users\JoshL\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\JoshL\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\JoshL\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i

name: XGBoost
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.50      0.08      0.14       183
           2       0.68      0.67      0.67      1383
           3       0.82      0.88      0.85      2433

    accuracy                           0.77      4000
   macro avg       0.50      0.41      0.42      4000
weighted avg       0.76      0.77      0.76      4000


Feature Importances:
year: 0.1860
genre: 0.1394
instrumentalness: 0.0865
duration_ms: 0.0517
Unnamed: 0.1: 0.0482
danceability: 0.0470
loudness: 0.0432
energy: 0.0410
speechiness: 0.0396
acousticness: 0.0376
time_signature: 0.0344
valence: 0.0342
mode: 0.0329
track_name: 0.0319
artist_name: 0.0318
liveness: 0.0305
tempo: 0.0287
track_id: 0.0282
key: 0.0273
Unnamed: 0: 0.0000
decade: 0.0000

Training CatBoost...


C:\Users\JoshL\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\JoshL\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\JoshL\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i

name: CatBoost
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.48      0.08      0.13       183
           2       0.68      0.66      0.67      1383
           3       0.82      0.88      0.85      2433

    accuracy                           0.77      4000
   macro avg       0.49      0.40      0.41      4000
weighted avg       0.75      0.77      0.75      4000


Feature Importances:
genre: 18.0536
instrumentalness: 7.5115
year: 6.6629
danceability: 5.7318
duration_ms: 5.3982
artist_name: 5.3321
speechiness: 4.9590
loudness: 4.8790
tempo: 4.5354
acousticness: 4.4964
energy: 4.2714
liveness: 4.1666
track_name: 4.1528
valence: 3.9720
track_id: 3.9655
Unnamed: 0: 3.3023
Unnamed: 0.1: 2.9603
key: 2.3992
decade: 1.7540
mode: 1.3191
time_signature: 0.1769

Training Logistic Regression...


C:\Users\JoshL\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\JoshL\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\JoshL\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i

name: Logistic Regression
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.07      0.21      0.10       183
           2       0.44      0.33      0.38      1383
           3       0.77      0.54      0.63      2433

    accuracy                           0.45      4000
   macro avg       0.32      0.27      0.28      4000
weighted avg       0.63      0.45      0.52      4000


Feature Importances:
Unnamed: 0.1: 7.1877
Unnamed: 0: 7.1877
loudness: 0.2546
key: 0.2169
instrumentalness: 0.0278
mode: 0.0202
tempo: 0.0202
decade: 0.0184
acousticness: 0.0151
time_signature: 0.0126
valence: 0.0079
speechiness: 0.0070
genre: 0.0067
danceability: 0.0048
energy: 0.0024
year: 0.0017
liveness: 0.0013
track_name: 0.0002
track_id: 0.0001
artist_name: 0.0001
duration_ms: 0.0000


C:\Users\JoshL\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [16]:
average_results = {name: np.mean(accs) for name, accs in all_results.items()}
avg_df = pd.DataFrame(average_results.items(), columns=["Model", "Average Accuracy"])
avg_df = avg_df.sort_values("Average Accuracy", ascending=False)

print("Average Accuracy for Each Model:")
print(avg_df)

Average Accuracy for Each Model:
                 Model  Average Accuracy
1              XGBoost           0.77025
2             CatBoost           0.76750
0        Random Forest           0.73375
3  Logistic Regression           0.45100
